In [ ]:
using DataFrames #for data wrangling
using StatsPlots #for plotting
using Turing #for MCMC
using CSV #CSV import
using Random #data generation (if required)

Import Data

In [ ]:
#this is a local document, but i've also saved a slightly cleaned up version in github with the same "valveTestRaw" title
valveData = CSV.read(raw"C:\Users\flemm\Documents\Grad School\MAE 298 (Bayesian Statistics)\Homework\Final Project\valveTestRaw.csv", DataFrame)

Row,Vendor,Population,Rejected,High Pop,% Over Set,% Over Limit,Set Pressure,Service
,String1,Int64,Int64,Int64,Float64,String7,Int64?,String15
1,a,9,2,1,0.1,4.3%,35,steam
2,b,13,0,0,0.0,0.0%,missing,air
3,c,85,1,1,0.13,9.7%,175,steam
4,d,108,21,4,0.05,1.5%,350,air
5,e,1,0,0,0.0,0.0%,missing,na
6,f,55,9,2,0.06,2.5%,400,liquid
7,g,41,5,2,0.07,3.3%,165,steam
8,h,2,0,0,0.0,0.0%,missing,air
9,z,40,6,1,0.24,20.0%,165,steam


Collect Number of Trials

In [16]:
vendors = (collect(valveData[:,1]))
n=0
for i in 1:length(vendors)
    n = valveData[i, 2] + n
end
n

560

Set Up Model

In [ ]:
@model function valveTesting(Vendor, Population, Rejects, highPops, serviceMedium)

    #hyperpriors
    #somehow, p should depend on the service medium... perhaps it could be normally distributed with a subtractive penalty based on service medium?
          
    #hyper prior
        μ ~ Beta(4, 2)
        σ ~ Expoential(1)
    

   #NOTE: INCLUSION OF 2 "M" VENDORS MESSES THIS UP FOR RIGHT NOW
    for i in 1:length(vendors)
        
        #prior
        p[i] ~ Normal(μ, σ)
        #if medium is air
            #some low penalty, exponential with λ = 1?
        #if medium is water
            #some higher penalty, exponential with λ = 0.5?
        #if medium is steam
            #some highest penalty, exponential with λ = 0.25?

        pass[i] = p[i] - mediumPenalty[i]

        #distribution of valves passing given pass probability p and n trials
        valvePass[i] ~ Binomial(n, pass[i])
    end
end

valveTesting (generic function with 2 methods)